## LOADER

In [10]:
import sys
from sys import platform
%load_ext autoreload
%autoreload 2

In [2]:
# if platform  == 'win32':
#     sys.argv=["_", r"P:\rmahajan14\ndte_data\muni_data", "20170101", "20190930"]
# if platform == 'linux' or platform == 'linux2':
#     def full_path(p):
#         return os.path.abspath(os.path.expanduser(p))
#     sys.argv=["_", r"/mnt/d/DiDi_Ride_Data/data", r"/mnt/d/DiDi_Ride_Data/Ride_Data_Cache"]
sys.argv=["_", "../Capstone - Data/", "../Ride_Data_Cache/"]

In [11]:
from loader1 import *
from utils import *
from common import *
# from sys import platform
import warnings
warnings.filterwarnings('ignore')

In [13]:
_ = load_all(override=True)

Processing 1 of 30 files
gps_20161103
order_20161103
Processing 2 of 30 files
gps_20161109
order_20161109
Processing 3 of 30 files
gps_20161124
order_20161124
Processing 4 of 30 files
gps_20161111
order_20161111
Processing 5 of 30 files
gps_20161113
order_20161113
Processing 6 of 30 files
gps_20161112
order_20161112
Processing 7 of 30 files
gps_20161117
order_20161117
Processing 8 of 30 files
gps_20161116
order_20161116
Processing 9 of 30 files
gps_20161102
order_20161102
Processing 10 of 30 files
gps_20161114
order_20161114
Processing 11 of 30 files
gps_20161101
order_20161101
Processing 12 of 30 files
gps_20161120
order_20161120
Processing 13 of 30 files
gps_20161125
order_20161125
Processing 14 of 30 files
gps_20161106
order_20161106
Processing 15 of 30 files
gps_20161127
order_20161127
Processing 16 of 30 files
gps_20161121
order_20161121
Processing 17 of 30 files
gps_20161128
order_20161128
Processing 18 of 30 files
gps_20161118
order_20161118
Processing 19 of 30 files
gps_2016112

## Process

In [14]:
# gps = read_data('gps')
orders = read_data('order', '20161101')

file path is ../Ride_Data_Cache/order_20161101.msgpack


In [18]:
# orders[orders.driver_id == '8bcb3fca2eb35d9deaffc274f005d2b8']

In [19]:
# dropping duplicates
orders = orders.drop_duplicates()
# gps = gps.drop_duplicates()

In [20]:
orders.head()

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,driver_id
0,00003ad9f837aca8ecdb9ae0729d0866,2016-11-01 14:35:44,2016-11-01 14:48:01,104.107101,30.699385,104.054980,30.688850,12.283333,64ee983f9979db09cb2586ec3893c467
1,0000c5bb7976b3dfe0fefa060265bdca,2016-11-01 15:06:22,2016-11-01 15:33:26,104.065170,30.702790,104.068280,30.666830,27.066667,19aa2c3cb2b05daf4ed06a3849c2c43c
2,0000f2c8aa21f2599a82923f317334bc,2016-11-01 19:38:49,2016-11-01 19:50:28,104.050859,30.687837,104.057100,30.662350,11.650000,15045dc2301012dda9cb8e98f4723efd
3,000102a3f8b31b3bc888572108888727,2016-11-01 11:22:02,2016-11-01 11:50:06,104.012450,30.622080,104.049190,30.755740,28.066667,53332bf0536f84453f0ab40a91774101
4,00017e395c1e449231b86f9149a793e5,2016-11-01 00:19:20,2016-11-01 00:34:41,104.092920,30.685830,103.990711,30.670803,15.350000,0dafbd548725f540988c5aa2554d782e


In [21]:
orders.shape

(181172, 9)

In [22]:
data = pd.DataFrame(orders.groupby('driver_id')['order_id'].count()).reset_index()
data.describe()

,order_id
count,35449.000000
mean,5.110779
std,3.983321
min,1.000000
25%,2.000000
50%,4.000000
75%,7.000000
max,56.000000


In [15]:
driver_ride_durations = orders.groupby('driver_id')[['ride_duration']].sum().reset_index()

In [16]:
driver_ride_durations.head()

,driver_id,ride_duration
0,0000131d486b69eb77ab6e9e7cca9f4c,75.583333
1,000211a97ba2b768ef941593bf1ae1a5,157.016667
2,000770ada4aa5d856b5a766fb6bf7cf1,87.900000
3,0007a380800445346fff19afc4071ce3,23.650000
4,0008c9d2ac91c00ef8b3ef80eba02a06,142.766667


### Single day total active time

In [ ]:
driver_day_min = pd.DataFrame(orders.groupby('driver_id')['ride_start_timestamp'].min()).reset_index()
driver_day_max = pd.DataFrame(orders.groupby('driver_id')['ride_stop_timestamp'].max()).reset_index()
driver_active_time = driver_day_min.merge(driver_day_max, on='driver_id', how='left')
driver_active_time['active_time'] = (driver_active_time['ride_stop_timestamp'] - driver_active_time['ride_start_timestamp']).dt.total_seconds() / 60
driver_active_time.head()

In [ ]:
driver_active_time[driver_active_time.driver_id == '310c5f3ffed8d3d42b90583c596cfd8e']

### Comparing total driver active time vs ride time

In [ ]:
# total driver active time
driver_stats = driver_active_time[['driver_id', 'active_time']].merge(driver_ride_durations, on='driver_id', how='left') 

# Filtering possible bad rows
# driver_stats = driver_stats[driver_stats.active_time > driver_stats.ride_duration]

driver_stats.describe()

In [ ]:
driver_stats[driver_stats.ride_duration > driver_stats.active_time]

In [ ]:
orders[orders.driver_id == '009ba30d83cca1e82e56473b0f029b3f']

In [ ]:
orders[orders.driver_id == '010af431d87d4b94869d350e52a9af15'].sort_values('ride_start_timestamp')

### Trying out geopandas plotting on orders table

In [ ]:
import geopandas
import matplotlib.pyplot as plt

In [ ]:
df = orders.copy()
df.head()

In [ ]:
# Converting to geopandas df
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.dropoff_longitude, df.dropoff_latitude))

In [ ]:
gdf.head()

In [ ]:
# Plotting on country level map
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
world

In [ ]:
# We restrict to China.
# ax = world[world.name == 'China'].plot(
#     color='white', edgecolor='black', figsize=(20,20))

# We can now plot our ``GeoDataFrame``.
# gdf.plot(ax=ax, color='red')

# plt.show()

#### Attempt 2 - Read china shapefile

In [ ]:
file = geopandas.read_file('../China_map/CHN_roads.dbf')

In [ ]:
gdf_driver = gdf[gdf.driver_id == '0373c98625e6d8e1fc56a4d61318138c']

In [ ]:
f, ax = plt.subplots(1, figsize=(80, 80))
ax = file.plot(axes=ax)
gdf_driver.plot(ax=ax, color='red')
ax.margins(x=-0, y=-0.25)
plt.show()

In [ ]:
# f, ax = plt.subplots(1, figsize=(80, 80))
# ax = file.plot(axes=ax)
# gdf_driver.plot(ax=ax, color='red')
# ax.margins(x=-0.4, y=-0.25)
# plt.show()

## GMAP plotting

In [ ]:
driver_latitude = gps[(gps.driver_id == '0373c98625e6d8e1fc56a4d61318138c') & 
                 (gps.order_id == 'eb9dd4095d9850e6287cefd813775a6c')].sort_values('timestamp').latitude
driver_longitude = gps[(gps.driver_id == '0373c98625e6d8e1fc56a4d61318138c') & 
                 (gps.order_id == 'eb9dd4095d9850e6287cefd813775a6c')].sort_values('timestamp').longitude

In [ ]:
gps[(gps.driver_id == '0373c98625e6d8e1fc56a4d61318138c') & 
                 (gps.order_id == 'eb9dd4095d9850e6287cefd813775a6c')].sort_values('timestamp')

In [ ]:
driver_latitude = list(driver_latitude)
driver_longitude = list(driver_longitude)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)

In [ ]:
import gmplot 
  
# latitude_list = [ 30.3358376, 30.307977, 30.3216419 ] 
# longitude_list = [ 77.8701919, 78.048457, 78.0413095 ] 
  
gmap5 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 

gmap5.marker(min_lat, min_lon, c='red')#, size=40, marker=True)
gmap5.marker(max_lat, max_lon, c='red')#, size=40, marker=True)

gmap5.plot( driver_latitude, driver_longitude, '# FF0000', 
                                size = 50, marker = False) 

gmap5.draw('map.html')
  
# polygon method Draw a polygon with 
# the help of coordinates 
# gmap5.polygon(latitude_list, longitude_list, 
#                    color = 'cornflowerblue') 

### Trying heatmap of snapshot of 9 AM rides

In [40]:
time = pd.to_datetime('2016-11-01 09:00:00')
first_orders = orders[['driver_id']].drop_duplicates()
first_orders['snapshot'] = time

In [41]:
first_orders

,driver_id,snapshot
105100,053c3c3156f27f1055d2516223036e29,2016-11-01 09:00:00
59575,3fe49726673176e92098de74a38d989c,2016-11-01 09:00:00
41744,44a773ba727b57b904612e27b3237eb6,2016-11-01 09:00:00
55455,e3dea9785be1822c7eb037ee058d2001,2016-11-01 09:00:00
150211,9c7983fed0240deed8d06c92b8569ee1,2016-11-01 09:00:00
...,...,...
101240,97ea843af908309ee67572ff9e01f6c7,2016-11-01 09:00:00
73398,3daa86de0f877a1ca039a6a8896c8a79,2016-11-01 09:00:00
23275,878b2658feee52e674870baedaee2852,2016-11-01 09:00:00
31914,526bba2ec3962f06c60414d660e00617,2016-11-01 09:00:00


In [42]:
# As of merge orders table to join first order pickups on or after 10 AM
## Sort right keys
orders = orders.sort_values('ride_start_timestamp')
df2 = pd.merge_asof(first_orders, orders, 
                    left_on='snapshot', 
                    right_on='ride_start_timestamp', 
                    direction='nearest',
                   by='driver_id')

cols = ['driver_id', 'order_id', 'snapshot', 'ride_start_timestamp',
        'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_longitude']

In [43]:
# Filter times which are at most + or - 5min from 6PM
import numpy as np
df2['time_delta'] = np.abs((df2.snapshot - df2.ride_start_timestamp)).dt.total_seconds() / 60
df2 = df2[df2.time_delta < 10]

In [48]:
df2.head()

,driver_id,snapshot,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,time_delta
5,0df0e6f5e603188c3e444028c354dbeb,2016-11-01 09:00:00,fda9df5b3dd9c585c54b304d0631a514,2016-11-01 09:06:05,2016-11-01 09:17:41,104.036800,30.681620,104.06332,30.67013,11.600000,6.083333
86,337d4d0e847dd8ad44c27346f80f256f,2016-11-01 09:00:00,ea3b29b8e4b335b5980e4984fd043320,2016-11-01 09:06:48,2016-11-01 09:20:59,104.114200,30.711220,104.15183,30.74553,14.183333,6.800000
116,9b67327c7f75483fd00021887e806fb3,2016-11-01 09:00:00,7abf27fba2139dcc6ee1dc7cbf9411ac,2016-11-01 09:07:31,2016-11-01 09:45:48,104.072701,30.695971,103.96979,30.61981,38.283333,7.516667
129,095fdb0052db44d416aa0ff971d2ec9d,2016-11-01 09:00:00,120d82ed07d94e5eafd80d949d239fdb,2016-11-01 08:53:08,2016-11-01 09:22:45,104.058540,30.644030,104.04559,30.71234,29.616667,6.866667
131,08704ff983c7d1d7affa106d91aa7e4b,2016-11-01 09:00:00,715edb8457eee71df9b41d853aa848b7,2016-11-01 09:09:44,2016-11-01 09:39:23,104.088640,30.755850,104.04332,30.68650,29.650000,9.733333


In [49]:
df2.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,time_delta
count,3181.000000,3181.000000,3181.000000,3181.000000,3181.000000,3181.000000
mean,104.073667,30.678780,104.072586,30.673817,21.172964,4.984507
std,0.044119,0.041697,0.038978,0.037309,12.299920,2.831506
min,103.822210,30.492120,103.863070,30.463230,0.083333,0.000000
25%,104.049390,30.656586,104.052600,30.656420,12.666667,2.566667
50%,104.073631,30.675020,104.072530,30.670020,18.600000,5.050000
75%,104.101712,30.698990,104.092130,30.692110,26.550000,7.350000
max,104.281570,30.978180,104.300050,30.989479,107.983333,9.983333


In [50]:
import gmplot 

driver_latitude = list(df2.pickup_latitude.values)
driver_longitude = list(df2.pickup_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)
  
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( driver_latitude, driver_longitude ) 
gmap4.draw('heatmap_9AM.html')


driver_latitude = list(df2.dropoff_latitude.values)
driver_longitude = list(df2.dropoff_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)
  
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( driver_latitude, driver_longitude ) 
gmap4.draw('heatmap_9AM_dropoff.html')

## Heatmap for 6PM

In [78]:
time = pd.to_datetime('2016-11-01 16:00:00')
last_orders = orders[['driver_id']].drop_duplicates()
last_orders['snapshot'] = time

In [72]:
last_orders.shape

(35449, 2)

In [79]:
# As of merge orders table to join last order pickups on or after 10 AM
## Sort right keys
orders = orders.sort_values('ride_start_timestamp')
df2 = pd.merge_asof(last_orders, orders, 
                    left_on='snapshot', 
                    right_on='ride_start_timestamp', 
                    direction='nearest',
                   by='driver_id')

cols = ['driver_id', 'order_id', 'snapshot', 'ride_start_timestamp',
        'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_longitude']

In [80]:
# Filter times which are at most + or - 5min from 6PM
import numpy as np
df2['time_delta'] = np.abs((df2.snapshot - df2.ride_start_timestamp)).dt.total_seconds() / 60
df2 = df2[df2.time_delta < 10]

In [81]:
df2.head()

,driver_id,snapshot,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,time_delta
15,377dd282929127a61ec20daffb42f50f,2016-11-01 16:00:00,e692393878a2a6ae4566847d84a9f756,2016-11-01 16:09:27,2016-11-01 16:22:37,104.099732,30.669089,104.07655,30.66004,13.166667,9.450000
20,cafc3505e52a72493bdeac60b0294263,2016-11-01 16:00:00,01c4a26f3ab08c117aac48168ef8232c,2016-11-01 15:53:00,2016-11-01 16:13:23,104.044960,30.681500,104.08684,30.66537,20.383333,7.000000
22,00d9c441b7bfbbf25fc4d0425081e014,2016-11-01 16:00:00,f941297b6efcb996280077a713944683,2016-11-01 16:05:56,2016-11-01 16:20:17,104.076530,30.688130,104.09281,30.68154,14.350000,5.933333
38,2fc936215fea3c9b3c5919b399b331c9,2016-11-01 16:00:00,35e0bde51cac0358270aca51ba881982,2016-11-01 15:55:00,2016-11-01 16:19:53,104.083366,30.559322,104.07380,30.65967,24.883333,5.000000
121,951c0736f20e5c222ec3fddec8a9ee1a,2016-11-01 16:00:00,739544e3c3597c034679f8090b98fafd,2016-11-01 16:06:07,2016-11-01 16:30:55,104.074580,30.660130,104.13935,30.67364,24.800000,6.116667


In [82]:
orders[orders.driver_id == 'cafc3505e52a72493bdeac60b0294263']

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,driver_id
53628,4c553d49d79def0cafcf782ae4aed7bf,2016-11-01 00:03:52,2016-11-01 00:09:44,104.097900,30.656890,104.077790,30.665870,5.866667,cafc3505e52a72493bdeac60b0294263
174115,f5c7af38a5ac3a3a81d0a03d3845e178,2016-11-01 00:13:38,2016-11-01 00:38:54,104.071370,30.667020,104.148630,30.591180,25.266667,cafc3505e52a72493bdeac60b0294263
85667,79d4cc5eafd4e858c5d61d86222a880e,2016-11-01 01:09:55,2016-11-01 01:21:44,104.087132,30.638442,104.048340,30.665330,11.816667,cafc3505e52a72493bdeac60b0294263
163813,e75b513e941b5d202fde0501ca777999,2016-11-01 01:23:40,2016-11-01 01:31:15,104.045020,30.668210,104.075630,30.663840,7.583333,cafc3505e52a72493bdeac60b0294263
42630,3c81ea85f4a01df9cf9c4ea05bee8598,2016-11-01 01:35:50,2016-11-01 01:56:26,104.070340,30.654530,104.166720,30.645310,20.600000,cafc3505e52a72493bdeac60b0294263
91642,82210d3da828622c0a28ea5770562262,2016-11-01 15:04:30,2016-11-01 15:29:02,104.103851,30.668957,104.045220,30.659180,24.533333,cafc3505e52a72493bdeac60b0294263
153717,d924babd7e2cfbc84a84d020cb7408f7,2016-11-01 15:32:38,2016-11-01 15:39:43,104.050972,30.662970,104.048230,30.682950,7.083333,cafc3505e52a72493bdeac60b0294263
159811,e1aa7584fdbc513fc51e29c283f4f71b,2016-11-01 15:49:25,2016-11-01 15:57:29,104.043630,30.680010,104.052600,30.663590,8.066667,cafc3505e52a72493bdeac60b0294263
1301,01c4a26f3ab08c117aac48168ef8232c,2016-11-01 15:53:00,2016-11-01 16:13:23,104.044960,30.681500,104.086840,30.665370,20.383333,cafc3505e52a72493bdeac60b0294263
128770,b61a481514a08e878c45d037aa9adfc0,2016-11-01 16:33:30,2016-11-01 16:55:51,104.096870,30.659320,104.052810,30.711150,22.350000,cafc3505e52a72493bdeac60b0294263


In [65]:
df2.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,time_delta
count,3481.000000,3481.000000,3481.000000,3481.000000,3481.000000,3481.000000
mean,104.071368,30.671607,104.070693,30.671664,23.512089,4.864809
std,0.040984,0.038769,0.045140,0.044437,14.448862,2.836042
min,103.747780,30.446443,103.828730,30.498090,0.366667,0.000000
25%,104.050800,30.654500,104.048440,30.652920,14.100000,2.416667
50%,104.072370,30.669110,104.072240,30.668740,20.700000,4.766667
75%,104.093580,30.690000,104.097170,30.692430,29.350000,7.300000
max,104.321130,30.884535,104.400600,31.334340,318.866667,9.983333


In [47]:
import gmplot 

driver_latitude = list(df2.pickup_latitude.values)
driver_longitude = list(df2.pickup_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)
  
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( driver_latitude, driver_longitude ) 
gmap4.draw('heatmap_6PM.html')


driver_latitude = list(df2.dropoff_latitude.values)
driver_longitude = list(df2.dropoff_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)
  
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( driver_latitude, driver_longitude ) 
gmap4.draw('heatmap_6PM_dropoff.html')

In [68]:
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
x = [104.046830] * 1000
y = [30.653985] * 1000
x = [np.random.uniform() * 10e-3 + x for x in list(df2.dropoff_latitude.values)]
y = [np.random.uniform() * 10e-3 + x for x in list(df2.dropoff_longitude.values)]
gmap4 = gmplot.GoogleMapPlotter(min(x) + (max(x) - min(x)) / 2, min(y) + (max(y) - min(y)) / 2, 12) 

gmap4.heatmap( x, y ) 
gmap4.draw('heatmap_test.html')

In [66]:
df2[['dropoff_latitude', 'dropoff_longitude']].drop_duplicates().shape

(2690, 2)

In [67]:
df2[['pickup_latitude', 'pickup_longitude']].drop_duplicates().shape

(3267, 2)

In [88]:
time = pd.to_datetime('2016-11-01 18:00:00')
last_orders = orders[['driver_id']].drop_duplicates()
last_orders['snapshot'] = time
# As of merge orders table to join last order pickups on or after 10 AM
## Sort right keys
orders = orders.sort_values('ride_start_timestamp')
df2 = pd.merge_asof(last_orders, orders, 
                    left_on='snapshot', 
                    right_on='ride_start_timestamp', 
                    direction='nearest',
                   by='driver_id')

cols = ['driver_id', 'order_id', 'snapshot', 'ride_start_timestamp',
        'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_longitude']
df2['time_delta'] = np.abs((df2.snapshot - df2.ride_start_timestamp)).dt.total_seconds() / 60
df2 = df2[df2.time_delta < 10]
print(df2[['dropoff_latitude', 'dropoff_longitude']].drop_duplicates().shape)
print(df2[['pickup_latitude', 'pickup_longitude']].drop_duplicates().shape)

(2667, 2)
(3331, 2)


In [94]:
tmp = df2[['dropoff_latitude', 'dropoff_longitude']].groupby(['dropoff_latitude', 'dropoff_longitude']).size().reset_index(name ='Count')
tmp = tmp[tmp.Count > 1].sort_values('Count', ascending=False)
driver_latitude = list(tmp.dropoff_latitude.values)
driver_longitude = list(tmp.dropoff_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)
  
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( driver_latitude, driver_longitude ) 
gmap4.draw('heatmap_test.html')

In [101]:
df2.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,time_delta
count,3462.000000,3462.000000,3462.000000,3462.000000,3462.000000,3462.000000
mean,104.073988,30.675887,104.071337,30.674676,24.797603,4.942880
std,0.040760,0.037894,0.042235,0.038871,14.159304,2.851327
min,103.881294,30.491898,103.777090,30.499210,0.100000,0.000000
25%,104.049889,30.655470,104.046830,30.653985,14.770833,2.500000
50%,104.074060,30.671909,104.072575,30.672090,22.150000,4.900000
75%,104.100015,30.694623,104.099350,30.695885,31.295833,7.400000
max,104.278809,30.943857,104.544680,30.940940,116.333333,9.983333


In [103]:
print(df2[['dropoff_latitude', 'dropoff_longitude']].drop_duplicates().shape)
print(df2[['pickup_latitude', 'pickup_longitude']].drop_duplicates().shape)

(2667, 2)
(3331, 2)


In [98]:
tmp.shape

(320, 3)

In [104]:
tmp.head(5)

,dropoff_latitude,dropoff_longitude,Count
2446,30.72096,104.03463,57
1822,30.68630,104.07403,24
1026,30.66359,104.05260,23
1192,30.66849,104.11203,16
665,30.65292,104.08220,13


In [43]:
min_lat, min_lon, max_lat, max_lon

(30.499209999999998, 103.77709, 30.94094, 104.54468)

In [52]:
driver_latitude = list(df2.pickup_latitude.values)
driver_longitude = list(df2.pickup_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)

In [45]:
min_lat, min_lon, max_lat, max_lon

(30.491898, 103.881294, 30.943857, 104.278809)

## Plotting all ride start locations

In [35]:
cols = ['driver_id', 'order_id', 'ride_start_timestamp', 'ride_stop_timestamp', 
        'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
driver_latitude = list(orders[cols].drop_duplicates().pickup_latitude.values)
driver_longitude = list(orders[cols].drop_duplicates().pickup_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)
  
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( driver_latitude, driver_longitude ) 
gmap4.draw('heatmap_all_pickup.html')

## Plotting all ride stop locations

In [36]:
cols = ['driver_id', 'order_id', 'ride_start_timestamp', 'ride_stop_timestamp', 
        'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
driver_latitude = list(orders[cols].drop_duplicates().dropoff_latitude.values)
driver_longitude = list(orders[cols].drop_duplicates().dropoff_longitude.values)
min_lat = min(driver_latitude)
min_lon = min(driver_longitude)
max_lat = max(driver_latitude)
max_lon = max(driver_longitude)
  
gmap4 = gmplot.GoogleMapPlotter(min_lat + (max_lat - min_lat) / 2, min_lon + (max_lon - min_lon) / 2, 12) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( driver_latitude, driver_longitude ) 
gmap4.draw('heatmap_all_dropoff.html')